In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [5]:
# Load the data
data = pd.read_csv("../../data/F10.7.csv")  # Replace "your_data.csv" with the path to your data file

# Preprocess the data
X = data[['Julian day', 'Carringtonrotation', 'Observed Flux', 'Adjusted Flux', 'URSI Flux']].values
y = data['Adjusted Flux'].values  # Adjusted Flux will be our target variable

# Normalize features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape data for CNN input (assuming 1D convolution)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [12]:
# Build the CNN model
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
    # MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)  # Output layer with one neuron for regression
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse')  # Using Mean Squared Error as loss function

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)


Epoch 1/10
518/518 [==============================] - 1s 1ms/step - loss: 2044.8704 - val_loss: 862.5042
Epoch 2/10
518/518 [==============================] - 1s 1ms/step - loss: 589.2459 - val_loss: 63.3954
Epoch 3/10
518/518 [==============================] - 1s 1ms/step - loss: 230.7088 - val_loss: 26.2242
Epoch 4/10
518/518 [==============================] - 1s 2ms/step - loss: 216.3708 - val_loss: 7.8246
Epoch 5/10
518/518 [==============================] - 1s 2ms/step - loss: 210.0078 - val_loss: 8.9940
Epoch 6/10
518/518 [==============================] - 1s 2ms/step - loss: 184.7605 - val_loss: 2.7500
Epoch 7/10
518/518 [==============================] - 1s 1ms/step - loss: 180.4444 - val_loss: 19.7087
Epoch 8/10
518/518 [==============================] - 1s 1ms/step - loss: 177.5464 - val_loss: 10.5826
Epoch 9/10
518/518 [==============================] - 1s 1ms/step - loss: 172.7446 - val_loss: 15.5612
Epoch 10/10
130/130 [==============================] - 0s 460us/step - los

In [11]:
# Define the future data point
next_julian_day = 2453309.354
next_carrington_rotation = 2022.656
next_observed_flux = 137.5
next_adjusted_flux = 135.7
next_ursi_flux = 121.5

# Predict future values
future_data = np.array([[next_julian_day, next_carrington_rotation, next_observed_flux, next_adjusted_flux, next_ursi_flux]])
future_data_scaled = scaler.transform(future_data)
future_data_reshaped = future_data_scaled.reshape((future_data_scaled.shape[0], future_data_scaled.shape[1], 1))
future_prediction = model.predict(future_data_reshaped)
print("Predicted Adjusted Flux for Future Time:", future_prediction)

1/1 [==============================] - 0s 79ms/step
Predicted Adjusted Flux for Future Time: [[123.37208]]
